In [7]:
import pandas as pd
import glob
import numpy as np

from dask.distributed import Client, LocalCluster
from dask import delayed


In [8]:

client

Client Scheduler: inproc://145.52.252.19/122670/1 Dashboard: http://localhost:43407/status,Cluster Workers: 1 Cores: 48 Memory: 67.18 GB


In [9]:
def dwelling_data_paths(sample_rate):
    """
    Reads in the file paths and dwelling id's of the combined smartmeter data.
    :return: file_paths, dwelling_ids, both as lists.
    """
    path = '//datc//opschaler//nan_information//'
    subscript = '_hour'

    if sample_rate == '10s':
        subscript = '_10s'
        
    complete_path = path+'*'+subscript+".csv"
    file_paths = np.array(glob.glob(complete_path))

    print('complete_path: '+complete_path)
    print('Detected %s smartmeter_data files.' % len(file_paths))
    
    dwelling_ids = np.array(list((map(lambda x: x[-20:-4], file_paths)))) 
    
    if sample_rate == '10s':
        #dwelling_ids = np.array(list((map(lambda x: x[-16:-8], file_paths)))) # 10s ids slicing
    
    
    return file_paths, dwelling_ids


def read_combined_df(path, dwelling_id):
    df = pd.read_csv(path, delimiter='\t', index_col=0)
    df = df.astype('float32')
    return df

In [10]:
paths, dwelling_ids = dwelling_data_paths('10s')

complete_path: //datc//opschaler//nan_information//*_10s.csv
Detected 56 smartmeter_data files.


In [11]:
def not_dask(sample_rate):
    paths, dwelling_ids = dwelling_data_paths(sample_rate)
    dfs_nan_table = []

    for i, path in enumerate(paths):
        dwelling_id = dwelling_ids[i]
        df = read_combined_df(path, dwelling_id)
        dfs_nan_table.append(df)
        
    final_df = pd.concat(dfs_nan_table)
    
    return final_df

In [13]:
not_dask("10s")

complete_path: //datc//opschaler//nan_information//*_10s.csv
Detected 56 smartmeter_data files.


ValueError: could not convert string to float: '2017-05-30 02:09:50'